# Read the original text from the book

In [ ]:
# Read the original text from the book

# Open the text file in read mode
file = open('/content/drive/MyDrive/anna/origin.txt', 'r')

# Read the entire content of the file
content = file.read()
test_text = """
ORIGINS


Born Eleanor Baker to a wealthy Quaker family in 1917 in Harrisburg, Pennsylvania, the future Nonie Wintour was a society girl. Her father, Ralph Baker, was a lawyer who left private practice to become a Harvard Law School professor. He specialized in trusts, and, before his death, established a substantial fund that would support his descendants, including his granddaughter Anna, over the course of many decades.
"""
# Close the file
file.close()

# Print the content
print(content[:100])
print(len(content))

Introduction


Of course, she was wearing the sunglasses.

Anna Wintour walked into the Vogue staff 
684435


In [ ]:
# divide the book into chapters
import re

# use regular expression to find chapter headers
pattern = r'\n\n(Chapter \d+)'

# split the text into chapters by headers
sections = re.split(pattern, content)

# Remove leading/trailing whitespace and group chapters with their content
chapters = [sections[i] + sections[i + 1] for i in range(0, len(sections) - 1, 2)]
print(chapters[1][:100])

 ORIGINS




Born Eleanor Baker to a wealthy Quaker family in 1917 in Harrisburg, Pennsylvania, the 


# Use BERT tokenizer

In [ ]:
# libraries
!pip install transformers 
from transformers import pipeline, BertTokenizer
# from transformers import AutoModelForTokenClassification, AutoTokenizer
import torch
import pickle

# model = AutoModelForTokenClassification.from_pretrained("bert-base-cased")
# tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
# Load the NER pipeline and tokenizer
nlp = pipeline("ner", model="dbmdz/bert-large-cased-finetuned-conll03-english")
tokenizer = BertTokenizer.from_pretrained("dbmdz/bert-large-cased-finetuned-conll03-english")


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 55.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 24.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 107.2 MB/s eta 0:00:00


In [ ]:
# tokenize the text into chunks of 512 tokens or less.
tokens = tokenizer.tokenize(content)
chunks = [tokens[i:i + 512] for i in range(0, len(tokens), 512)]

# initialize an empty list to store the entities
entities=[]

# Process each chunk
for chunk in chunks:
    chunk_text = tokenizer.convert_tokens_to_string(chunk)
    output = nlp(chunk_text)
    entities.extend(output)

In [ ]:
# Save the entities to a file
with open("/content/drive/MyDrive/anna/entities.pkl", "wb") as file:
    pickle.dump(entities, file)

In [ ]:
# combine consecutive words if they refer to the same objects.
combined_entities = []
entity_group = None

for entity in entities:
    if entity_group is None:
        entity_group = entity
    elif (entity['start'] == entity_group['end']) and (entity['entity'] == entity_group['entity']):
        entity_group['end'] = entity['end']
        entity_group['word'] += entity['word'].replace('##', '')  # Remove '##'
        entity_group['score'] = (entity_group['score'] + entity['score']) / 2
    elif (entity['start'] - entity_group['end'] == 1) and (entity['entity'] == entity_group['entity']):
        entity_group['end'] = entity['end']
        entity_group['word'] += ' ' + entity['word']  # Add space before the word
        entity_group['score'] = (entity_group['score'] + entity['score']) / 2
    else:
        combined_entities.append(entity_group)
        entity_group = entity

if entity_group is not None:  # Add the last entity_group to the list
    combined_entities.append(entity_group)


In [ ]:
# Save the entities to a file
with open("/content/drive/MyDrive/anna/combined_entities.pkl", "wb") as file:
    pickle.dump(combined_entities, file)

# try  `flair/ner-english-ontonotes `

In [ ]:
# import libraries
#!pip install flair

from flair.data import Sentence
from flair.models import SequenceTagger

# load tagger
#tagger = SequenceTagger.load("flair/ner-english-ontonotes")

In [ ]:
from flair.nn import Classifier
from flair.splitter import SegtokSentenceSplitter

# example text with many sentences
# text = "This is a sentence. This is another sentence. I love Berlin last year."

# initialize sentence splitter
splitter = SegtokSentenceSplitter()

# use splitter to split text into list of sentences
sentences = splitter.split(content)

# predict tags for sentences
tagger = Classifier.load('ner-ontonotes-fast')
tagger.predict(sentences)

# iterate through sentences and print predicted labels
# for sentence in sentences:
#    print(sentence)

2023-05-29 15:23:27,850 SequenceTagger predicts: Dictionary with 75 tags: O, S-PERSON, B-PERSON, E-PERSON, I-PERSON, S-GPE, B-GPE, E-GPE, I-GPE, S-ORG, B-ORG, E-ORG, I-ORG, S-DATE, B-DATE, E-DATE, I-DATE, S-CARDINAL, B-CARDINAL, E-CARDINAL, I-CARDINAL, S-NORP, B-NORP, E-NORP, I-NORP, S-MONEY, B-MONEY, E-MONEY, I-MONEY, S-PERCENT, B-PERCENT, E-PERCENT, I-PERCENT, S-ORDINAL, B-ORDINAL, E-ORDINAL, I-ORDINAL, S-LOC, B-LOC, E-LOC, I-LOC, S-TIME, B-TIME, E-TIME, I-TIME, S-WORK_OF_ART, B-WORK_OF_ART, E-WORK_OF_ART, I-WORK_OF_ART, S-FAC


In [ ]:
for sentence in sentences[:10]:
  # print(sentence.to_dict(tag_type='ner'))
  print(sentence.get_label('ner'))
  

Sentence[1]: "Introduction" → O (1.0)
Sentence[9]: "Of course, she was wearing the sunglasses." → O (1.0)
Span[0:2]: "Anna Wintour" → PERSON (0.9335)
Span[8:10]: "the night" → TIME (0.8935)
Sentence[12]: "Others had just been up crying, scared, in shock." → O (1.0)
Span[0:1]: "Anna" → PERSON (0.9803)
Span[2:6]: "November 9, 2016" → DATE (0.736)
Span[1:3]: "Hillary Clinton" → PERSON (0.9486)
Span[3:5]: "5:00 a.m." → TIME (0.8083)
Span[16:17]: "Anna" → PERSON (0.9909)


## save NERs in json

In [ ]:
# save NERs in json
import json

data = []
for i, sentence in enumerate(sentences):
    entities = sentence.get_spans('ner')
    for entity in entities:
        data.append({
            'entity': entity.text,
            'value': entity.tag,
            'score': entity.score,
            'start_pos': entity.start_position,
            'sentence_num': i
        })

with open('/content/drive/MyDrive/anna/entities.json', 'w') as f:
    json.dump(data, f, indent=4)


# entity linking

In [ ]:
## Entity linking

# predict tags for sentences
linker_tagger = Classifier.load('linker')
linker_tagger.predict(sentences)

2023-05-28 18:41:54,667 https://nlp.informatik.hu-berlin.de/resources/models/zelda/v2/zelda-v2.pt not found in cache, downloading to /tmp/tmppqbc4r6b


100%|██████████| 1.76G/1.76G [02:26<00:00, 12.9MB/s]

2023-05-28 18:44:21,434 copying /tmp/tmppqbc4r6b to cache at /root/.flair/models/zelda-v2.pt


2023-05-28 18:44:37,983 removing temp file /tmp/tmppqbc4r6b
2023-05-28 18:45:15,340 SequenceTagger predicts: Dictionary with 20 tags: <unk>, O, S-ORG, S-MISC, B-PER, E-PER, S-LOC, B-ORG, E-ORG, I-PER, S-PER, B-MISC, I-MISC, E-MISC, I-ORG, B-LOC, E-LOC, I-LOC, <START>, <STOP>


In [ ]:
sentences[:10]

[Sentence[1]: "Introduction",
 Sentence[9]: "Of course, she was wearing the sunglasses.",
 Sentence[25]: "Anna Wintour walked into the Vogue staff meeting and looked at the group that had gathered around the table about ten thirty that morning." → ["Anna Wintour"/Anna_Wintour, "Vogue"/Vogue_(magazine)],
 Sentence[21]: "Many of them had been working late into the night, coming up with stories attempting to explain the unprecedented.",
 Sentence[12]: "Others had just been up crying, scared, in shock.",
 Sentence[22]: "Anna had extraordinary influence over a great many things, but the outcome of this election was not one of them." → ["Anna"/<unk>],
 Sentence[7]: "It was November 9, 2016.",
 Sentence[35]: "Despite Hillary Clinton’s loss, after Vogue’s full-throttle support, including an endorsement—the first of its kind in the magazine’s 124-year history—Anna started the day as usual." → ["Hillary Clinton"/Hillary_Clinton, "Vogue"/Vogue_(magazine), "Anna"/<unk>],
 Sentence[82]: "She rose b